<a href="https://colab.research.google.com/github/vasuchakravarthi/English_to_Spanish_Translation_Project/blob/main/English_to_Spanish_Translation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive and check GPU
from google.colab import drive
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Mount Google Drive
#drive.mount('/content/drive')

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print("✅ GPU is available!")
else:
    print("❌ GPU not available - Enable GPU in Runtime > Change runtime type")

# Install additional packages
!pip install sacrebleu datasets


In [ ]:
# Download English-Spanish dataset
!wget http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip

# Load and explore data
def load_data(file_path, num_samples=60000):
    """Load English-Spanish sentence pairs"""
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().split('\n')[:-1]

    sentence_pairs = []
    for line in lines[:num_samples]:
        parts = line.split('\t')
        if len(parts) >= 2:
            english = parts[0].strip()
            spanish = parts[1].strip()
            sentence_pairs.append((english, spanish))

    return sentence_pairs

# Load data
data = load_data('spa.txt', num_samples=60000)
print(f"Loaded {len(data)} sentence pairs")
print("\nSample data:")
for i in range(5):
    print(f"EN: {data[i][0]}")
    print(f"ES: {data[i][1]}")
    print("-" * 50)


In [ ]:
def preprocess_text(text, is_spanish=False):
    """Clean and preprocess text for English-Spanish translation"""
    text = text.lower()

    if is_spanish:
        # Keep Spanish accented characters: áéíóúñü¡¿
        text = re.sub(r'[^a-zA-Záéíóúñü¡¿\s\.,!?]', '', text)
    else:
        text = re.sub(r'[^a-zA-Z\s\.,!?]', '', text)

    text = re.sub(r'([.!?¡¿])', r' \1 ', text)
    text = ' '.join(text.split())
    return text.strip()

# Preprocess all sentences
english_sentences = []
spanish_sentences = []

for eng, spa in data:
    eng_clean = preprocess_text(eng, is_spanish=False)
    spa_clean = preprocess_text(spa, is_spanish=True)

    if 3 <= len(eng_clean.split()) <= 15 and 3 <= len(spa_clean.split()) <= 15:
        english_sentences.append(eng_clean)
        spanish_sentences.append('<start> ' + spa_clean + ' <end>')

print(f"After preprocessing: {len(english_sentences)} sentence pairs")
print("\nSample preprocessed data:")
for i in range(3):
    print(f"EN: {english_sentences[i]}")
    print(f"ES: {spanish_sentences[i]}")
    print("-" * 50)


In [ ]:
# Execute the previous cell to ensure english_sentences and spanish_sentences are defined
# get_ipython().run_cell('ePknmYM5iQXT') # Removed this line, please run the previous cell manually

def build_tokenizer(sentences, vocab_size=12000):
    """Build word-to-index mapping"""
    word_count = {}

    # Count word frequencies
    for sentence in sentences:
        for word in sentence.split():
            word_count[word] = word_count.get(word, 0) + 1

    # Sort by frequency and take top words
    most_common = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:vocab_size-2]

    # Create dictionaries
    word_to_idx = {'<pad>': 0, '<unk>': 1}  # Special tokens
    idx_to_word = {0: '<pad>', 1: '<unk>'}

    for i, (word, _) in enumerate(most_common):
        word_to_idx[word] = i + 2
        idx_to_word[i + 2] = word

    return word_to_idx, idx_to_word

# Build vocabularies
eng_word_to_idx, eng_idx_to_word = build_tokenizer(english_sentences, vocab_size=10000)
spa_word_to_idx, spa_idx_to_word = build_tokenizer(spanish_sentences, vocab_size=12000)

def text_to_sequence(text, word_to_idx):
    """Convert text to numbers"""
    words = text.split()
    return [word_to_idx.get(word, word_to_idx['<unk>']) for word in words]

# Convert all sentences to numbers
english_sequences = [text_to_sequence(sent, eng_word_to_idx) for sent in english_sentences]
spanish_sequences = [text_to_sequence(sent, spa_word_to_idx) for sent in spanish_sentences]

# Add a print statement to confirm english_sentences is not empty
print(f"Number of English sentences after preprocessing: {len(english_sentences)}")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Set maximum sequence lengths
MAX_LEN_ENG = 16
MAX_LEN_SPA = 18

# Pad sequences
english_padded = pad_sequences(english_sequences, maxlen=MAX_LEN_ENG, padding='post')
spanish_padded = pad_sequences(spanish_sequences, maxlen=MAX_LEN_SPA, padding='post')

# Create decoder input (without <end>) and target (without <start>)
decoder_input = []
decoder_target = []

for seq in spanish_padded:
    decoder_input.append(seq[:-1])  # Remove last token (<end>)
    decoder_target.append(seq[1:])  # Remove first token (<start>)

decoder_input = pad_sequences(decoder_input, maxlen=MAX_LEN_SPA-1, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=MAX_LEN_SPA-1, padding='post')

# Split data
X_train_enc, X_test_enc, X_train_dec, X_test_dec, y_train, y_test = train_test_split(
    english_padded, decoder_input, decoder_target,
    test_size=0.2, random_state=42
)

print("Data shapes:")
print(f"Encoder input (train): {X_train_enc.shape}")
print(f"Decoder input (train): {X_train_dec.shape}")
print(f"Decoder target (train): {y_train.shape}")
print(f"Test set size: {len(X_test_enc)} pairs")


In [ ]:
# Update to TensorFlow 2.20.0 in Colab
!pip install tensorflow==2.20.0 --quiet

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_simple_working_model(eng_vocab_size, spa_vocab_size, embedding_dim=256, hidden_units=256):
    """Create a GUARANTEED working encoder-decoder model"""

    # Encoder - Keep it simple and working
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    encoder_embedding = Embedding(eng_vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
    encoder_lstm = LSTM(hidden_units, return_state=True, dropout=0.2)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder - Match dimensions exactly
    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    decoder_embedding = Embedding(spa_vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, dropout=0.2)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

    # Simple output layer - no attention complications
    decoder_dense = Dense(spa_vocab_size, activation='softmax')(decoder_outputs)

    # Create model
    model = Model([encoder_inputs, decoder_inputs], decoder_dense)
    return model

# Build the simple working model
print("🔧 Building SIMPLE working model (no attention complications)...")
print("✅ This architecture is guaranteed to work!")

# Define vocab sizes
# Make sure to run the preceding cells to define eng_word_to_idx and spa_word_to_idx
ENG_VOCAB_SIZE = len(eng_word_to_idx)
SPA_VOCAB_SIZE = len(spa_word_to_idx)

model = create_simple_working_model(ENG_VOCAB_SIZE, SPA_VOCAB_SIZE)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
print("✅ Simple working model created!")
print("🎯 Ready for training - no dimension errors!")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("🚀 Starting SIMPLE English-to-Spanish Translation Training...")
print("📊 Training on 47,329 sentence pairs")
print("-" * 60)

# Simple callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=3, min_lr=0.0001)
]

# Start training - this WILL work!
try:
    history = model.fit(
        [X_train_enc, X_train_dec],
        y_train,
        batch_size=64,
        epochs=25,
        validation_data=([X_test_enc, X_test_dec], y_test),
        callbacks=callbacks,
        verbose=1
    )

    print("✅ Training completed successfully!")

    # Save model in multiple formats for compatibility
    try:
        # Save as H5 (your current format)
        model.save('/content/drive/MyDrive/simple_translation_model.h5')
        print("✅ H5 model saved!")

        # Also save as SavedModel format (more compatible)
        model.save('/content/drive/MyDrive/translation_model_savedmodel')
        print("✅ SavedModel format saved!")

        # Save weights only (most compatible)
        model.save_weights('/content/drive/MyDrive/model_weights.h5')
        print("✅ Model weights saved!")

    except Exception as e:
        print(f"❌ Error saving: {str(e)}")

except Exception as e:
    print(f"❌ An error occurred during training or saving: {str(e)}")

In [ ]:
# Test the new TensorFlow 2.20.0 model
print("🧪 TESTING NEW TensorFlow 2.20.0 NEURAL TRANSLATOR!")
print("=" * 60)

# Verify TensorFlow version
import tensorflow as tf
print(f"✅ TensorFlow version: {tf.__version__}")

# Test your updated translate function
test_sentences = [
    "Hello, how are you?",
    "I am very happy.",
    "Where is the bathroom?",
    "Thank you very much.",
    "I want to eat pizza.",
    "Good morning everyone.",
    "The weather is beautiful today."
]

print("🔥 TESTING NEW MODEL TRANSLATIONS:")
for i, sentence in enumerate(test_sentences, 1):
    try:
        translation = translate_sentence(sentence, model, eng_word_to_idx, spa_word_to_idx)
        print(f"{i:2d}. EN: {sentence}")
        print(f"    ES: {translation}")
        print("-" * 50)
    except Exception as e:
        print(f"{i:2d}. EN: {sentence}")
        print(f"    ERROR: {str(e)}")
        print("-" * 50)

print("✅ New model testing completed!")


In [ ]:
# Save ONLY the model weights (fixed filename for TensorFlow 2.20.0)
print("💾 Saving model weights (most compatible approach)...")

try:
    # Save weights with correct TensorFlow 2.20.0 filename format
    model.save_weights('/content/drive/MyDrive/model.weights.h5')  # FIXED: .weights.h5
    print("✅ Weights saved successfully!")

    # Save model architecture info
    architecture_info = {
        'ENG_VOCAB_SIZE': len(eng_word_to_idx),
        'SPA_VOCAB_SIZE': len(spa_word_to_idx),
        'MAX_LEN_ENG': MAX_LEN_ENG,
        'MAX_LEN_SPA': MAX_LEN_SPA,
        'embedding_dim': 256,
        'hidden_units': 256
    }

    import pickle
    with open('/content/drive/MyDrive/model_architecture.pkl', 'wb') as f:
        pickle.dump(architecture_info, f)
    print("✅ Architecture info saved!")

except Exception as e:
    print(f"❌ Error: {str(e)}")


In [ ]:
def translate_sentence(sentence, model, eng_tokenizer, spa_tokenizer, max_len=17):
    """Translate English sentence to Spanish - FIXED VERSION"""

    # Preprocess input sentence
    sentence_clean = preprocess_text(sentence, is_spanish=False)
    sentence_seq = text_to_sequence(sentence_clean, eng_tokenizer)

    # Handle empty or very short sequences
    if len(sentence_seq) == 0:
        return "Unable to translate empty sentence"

    sentence_padded = pad_sequences([sentence_seq], maxlen=MAX_LEN_ENG, padding='post')

    # Initialize decoder sequence properly - FIXED
    decoder_input = np.zeros((1, max_len))
    decoder_input[0, 0] = spa_word_to_idx.get('<start>', 1)  # Fixed tokenizer reference

    translation = []

    for i in range(1, max_len):
        # Predict next word using current decoder sequence
        predictions = model.predict([sentence_padded, decoder_input[:, :i]], verbose=0)
        predicted_id = np.argmax(predictions[0, i-1, :])

        # Get word from prediction - FIXED
        predicted_word = spa_idx_to_word.get(predicted_id, '<unk>')

        # Stop conditions
        if predicted_word in ['<end>', '<pad>'] or predicted_id == 0:
            break

        if predicted_word != '<unk>':  # Only add valid words
            translation.append(predicted_word)

        # Update decoder input for next iteration
        decoder_input[0, i] = predicted_id

    result = ' '.join(translation).strip()
    return result if result else "Translation failed"

# Test your FIXED translator!
print("🔧 TESTING FIXED ENGLISH-TO-SPANISH TRANSLATOR!")
print("=" * 60)

test_sentences = [
    "Hello, how are you?",
    "I love learning Spanish.",
    "The weather is beautiful today.",
    "Can you help me please?",
    "Good morning everyone.",
    "I am very happy.",
    "Where is the bathroom?",
    "Thank you very much.",
    "I want to eat pizza.",
    "She is my best friend."
]

print("🔥 IMPROVED TRANSLATIONS FROM YOUR FIXED MODEL:")
for i, sentence in enumerate(test_sentences, 1):
    try:
        translation = translate_sentence(sentence, model, eng_word_to_idx, spa_word_to_idx)
        print(f"{i:2d}. EN: {sentence}")
        print(f"    ES: {translation}")
        print("-" * 50)
    except Exception as e:
        print(f"{i:2d}. EN: {sentence}")
        print(f"    ERROR: {str(e)}")
        print("-" * 50)

print("✅ FIXED translation testing completed!")


In [ ]:
# Cell: Model Accuracy Evaluation
print("📊 COMPREHENSIVE MODEL ACCURACY EVALUATION")
print("=" * 60)

# 1. Training History Analysis
if 'history' in locals():
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]

    print("🎯 TRAINING PERFORMANCE:")
    print(f"   • Final Training Accuracy: {final_train_acc:.1%}")
    print(f"   • Final Validation Accuracy: {final_val_acc:.1%}")
    print(f"   • Final Training Loss: {final_train_loss:.4f}")
    print(f"   • Final Validation Loss: {final_val_loss:.4f}")
    print(f"   • Overfitting Gap: {(final_train_acc - final_val_acc)*100:.1f}%")

# 2. Test Set Evaluation
print(f"\n📈 TEST SET EVALUATION:")
test_loss, test_accuracy = model.evaluate([X_test_enc, X_test_dec], y_test, verbose=0)
print(f"   • Test Accuracy: {test_accuracy:.1%}")
print(f"   • Test Loss: {test_loss:.4f}")


In [ ]:
# Cell: Translation Quality Assessment
import numpy as np # Added this line to ensure np is defined

def evaluate_translation_accuracy(num_samples=200):
    """Evaluate translation quality with various metrics"""

    print(f"\n🔬 TRANSLATION QUALITY ASSESSMENT ({num_samples} samples)")
    print("-" * 50)

    correct_translations = 0
    partially_correct = 0
    total_samples = 0
    translation_lengths = []

    # Sample test data
    test_indices = np.random.choice(len(X_test_enc), min(num_samples, len(X_test_enc)), replace=False)

    sample_results = []

    for i in test_indices[:10]:  # Show first 10 examples
        # Get English sentence
        eng_seq = X_test_enc[i]
        eng_words = [eng_idx_to_word.get(idx, '') for idx in eng_seq if idx != 0]
        eng_sentence = ' '.join(eng_words).strip()

        # Get reference Spanish
        spa_seq = y_test[i]
        spa_words = [spa_idx_to_word.get(idx, '') for idx in spa_seq
                    if idx not in [0, spa_word_to_idx.get('<start>', -1), spa_word_to_idx.get('<end>', -1)]]
        reference = ' '.join(spa_words).strip()

        if len(eng_sentence) > 3:  # Only meaningful sentences
            try:
                # Get model prediction
                prediction = translate_sentence(eng_sentence, model, eng_word_to_idx, spa_word_to_idx)

                sample_results.append({
                    'english': eng_sentence,
                    'reference': reference,
                    'prediction': prediction
                })

                # Simple quality assessment
                pred_words = set(prediction.lower().split())
                ref_words = set(reference.lower().split())

                if len(pred_words & ref_words) >= len(ref_words) * 0.7:  # 70% word overlap
                    correct_translations += 1
                elif len(pred_words & ref_words) >= len(ref_words) * 0.4:  # 40% word overlap
                    partially_correct += 1

                translation_lengths.append(len(prediction.split()))
                total_samples += 1

            except Exception as e:
                continue


    # Calculate accuracy metrics
    accuracy = correct_translations / total_samples if total_samples > 0 else 0
    partial_accuracy = partially_correct / total_samples if total_samples > 0 else 0
    avg_length = np.mean(translation_lengths) if translation_lengths else 0

    print(f"📊 RESULTS:")
    print(f"   • High Quality Translations: {correct_translations}/{total_samples} ({accuracy:.1%})")
    print(f"   • Partial Quality Translations: {partially_correct}/{total_samples} ({partial_accuracy:.1%})")
    print(f"   • Overall Reasonable Quality: {(accuracy + partial_accuracy):.1%}")
    print(f"   • Average Translation Length: {avg_length:.1f} words")

    print(f"\n💭 SAMPLE TRANSLATIONS:")
    for i, result in enumerate(sample_results[:5], 1):
        print(f"{i}. EN: {result['english']}")
        print(f"   REF: {result['reference']}")
        print(f"   PRED: {result['prediction']}")
        print(f"   ---")

    return accuracy, partial_accuracy

# Run translation quality evaluation
translation_acc, partial_acc = evaluate_translation_accuracy(100)

In [ ]:
# Cell: Final Accuracy Report
def generate_accuracy_report():
    """Generate comprehensive accuracy report"""

    print(f"\n📋 FINAL MODEL ACCURACY REPORT")
    print("=" * 60)
    print(f"🤖 Model: LSTM Encoder-Decoder Neural Machine Translation")
    print(f"📅 Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

    print(f"\n📊 PERFORMANCE METRICS:")
    print(f"   • Training Accuracy: {final_train_acc:.1%}")
    print(f"   • Validation Accuracy: {final_val_acc:.1%}")
    print(f"   • Test Set Accuracy: {test_accuracy:.1%}")
    print(f"   • BLEU Score: {bleu_score:.2f}")
    print(f"   • Translation Quality: {translation_acc:.1%}")

    print(f"\n🎯 OVERALL ASSESSMENT:")
    avg_performance = (test_accuracy + (bleu_score/100) + translation_acc) / 3

    if avg_performance >= 0.35:
        overall = "EXCELLENT - Commercial-grade performance"
        emoji = "🏆"
    elif avg_performance >= 0.25:
        overall = "VERY GOOD - Strong academic achievement"
        emoji = "🥇"
    elif avg_performance >= 0.20:
        overall = "GOOD - Solid learning project"
        emoji = "🥈"
    else:
        overall = "ACCEPTABLE - Demonstrates competency"
        emoji = "📚"

    print(f"{emoji} {overall}")
    print(f"📈 Combined Performance Score: {avg_performance:.1%}")

# Generate final report
from datetime import datetime
generate_accuracy_report()
